In [1]:
import numpy as np
import pandas as pd
import torch
import torchvision
from torchvision import transforms
from PIL import Image
from sklearn.neighbors import NearestNeighbors
import joblib

!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-py3-none-any.whl size=16035 sha256=b29e0f0ffc0d31e2e9137861092c7fc1de08d13495cb6a3b723f28df09252f71
  Stored in directory: /root/.cache/pip/wheels/b7/cc/0d/41d384b0071c6f46e542aded5f8571700ace4f1eb3f1591c29
Successfully built efficientnet-pytorch


In [2]:
from pathlib import Path

In [3]:
LIB_DIR=Path('/kaggle/input/digix-gallery/gallery/')
MODEL_WEIGHTS= Path('/kaggle/input/digix-ai-1st-attempt/eff_net_w_2.pt')
ball_tree_dump_file = 'library_ball_tree.sav'
lib_files_dump_file = 'library_files_list.sav'
RESCALE_SIZE=224

In [4]:
eval_on_gpu = torch.cuda.is_available()

if eval_on_gpu:
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

In [5]:
lib_files = sorted(list(LIB_DIR.rglob('*.jpg')))

In [6]:
from torch.utils.data import Dataset, DataLoader

In [7]:
class GoodsDataset(Dataset):
    def __init__(self, files):
        super().__init__()
        # список файлов для загрузки
        self.files = sorted(files)

        self.len_ = len(self.files)
            
    def __len__(self):
        return self.len_
      
    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image
    
    def __getitem__(self, index):
        # для преобразования изображений в тензоры PyTorch и нормализации входа
        data_transforms = transforms.Compose([
            transforms.Resize(RESCALE_SIZE),
            transforms.CenterCrop(RESCALE_SIZE),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        x = self.load_sample(self.files[index])
        if (len(x.mode) < 2):
            x = transforms.Grayscale(3)(x)
            
        transforms.functional.adjust_saturation(img=x,saturation_factor=1.25)
        transforms.functional.adjust_gamma(img=x, gamma=0.25)
        
        x = data_transforms(x)
        return x

In [8]:
lib_dataset = GoodsDataset(lib_files)

In [9]:
import torch.nn as nn

In [10]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_name('efficientnet-b1')

additional_ftrs = 4096
n_classes = 3094
#Изменяем выходные слои модели
num_ftrs_resnext = model._fc.in_features
new_fc_seq = nn.Sequential(
    nn.Linear(num_ftrs_resnext,additional_ftrs),
    nn.LeakyReLU(),
    nn.Linear(additional_ftrs, n_classes)
)
model._fc = new_fc_seq

model.load_state_dict(torch.load(MODEL_WEIGHTS))

<All keys matched successfully>

In [11]:
def library_processing(model, dataset, batch_size):
    feature_vector = np.array(list())
    
    model.eval()
    
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    with torch.no_grad():
        for inputs in data_loader:
            inputs = inputs.to(DEVICE)
            outputs = model(inputs).cpu()
            if len(feature_vector) ==0:
                feature_vector=np.array(outputs)
            else:
                np.concatenate([feature_vector, np.array(outputs)], axis=0)
            
    return feature_vector
            

In [12]:
def prepare_model(model):
    fc_without_last_layer = list(model._fc.children())[:-2]
    model._fc = torch.nn.Sequential(*fc_without_last_layer)

In [13]:
model

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [14]:
for p, name in model.named_parameters():
    print(name)

Parameter containing:
tensor([[[[-6.3620e-01, -1.5379e-01,  1.5169e-01],
          [-1.9757e-01, -6.7668e-02,  1.0308e-02],
          [ 3.4911e-02, -6.3531e-03,  1.9413e-02]],

         [[-1.2618e+00, -5.1097e-01,  2.5009e-01],
          [-3.6755e-01, -2.3591e-01,  1.6008e-02],
          [ 9.5980e-03,  5.4578e-02,  4.7310e-02]],

         [[-3.2306e-01, -1.0334e-01,  6.8485e-02],
          [-9.1873e-02, -2.5816e-02, -9.8065e-03],
          [ 3.3706e-02, -9.9613e-03, -5.0335e-03]]],


        [[[ 1.7601e-01,  4.4765e-02,  2.0709e-02],
          [ 3.9302e-01,  1.7437e-01,  9.4149e-02],
          [ 1.8246e-01,  6.1758e-02, -2.2089e-02]],

         [[ 7.5235e-01,  2.7725e-01, -6.7544e-04],
          [ 1.0594e+00,  5.0854e-01,  1.3760e-01],
          [ 2.3990e-01,  3.8417e-02, -5.9950e-02]],

         [[ 8.0627e-02,  5.5685e-02,  1.6516e-02],
          [ 1.8565e-01,  5.9905e-02,  5.1997e-02],
          [ 9.2310e-02,  5.3457e-02, -2.1884e-02]]],


        [[[ 3.3887e-02, -5.7342e-02,  4.9630

In [15]:
prepare_model(model)

model.to(DEVICE)
lib_feature_vector = library_processing(model, lib_dataset, batch_size=64)

lib_tree = NearestNeighbors(n_neighbors=10, metric="cosine")
lib_tree.fit(lib_feature_vector)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [16]:
joblib.dump(lib_tree, ball_tree_dump_file)

joblib.dump(lib_files, lib_files_dump_file)

['library_files_list.sav']